In [61]:
#dataset creation of images
import cv2
import os

# Set the directory where you'll save images (create separate folders for each item)
item_name = "Lays_yellow"  # Change this to your item's name
save_dir = f"Dataset_items/{item_name}"

# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Set parameters
img_count = 0
max_images = 100  # Capture 100 images
start_capture = False  # Flag to start capturing images

print(f"Press 's' to start capturing images of {item_name}.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Define region of interest (bounding box)
    # You can change these values to fit the size of the object you're capturing
    (h, w) = frame.shape[:2]
    box_x = int(w * 0.3)
    box_y = int(h * 0.3)
    box_w = int(w * 0.4)
    box_h = int(h * 0.4)
    
    # Draw a rectangle around the region of interest (ROI)
    cv2.rectangle(frame, (box_x, box_y), (box_x + box_w, box_y + box_h), (0, 255, 0), 2)
    
    # Display instructions on the frame
    if not start_capture:
        cv2.putText(frame, f"Press 's' to start capturing images", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        cv2.putText(frame, f"Capturing... {img_count}/{max_images} images", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow("Object Capture", frame)

    # Listen for keypress
    key = cv2.waitKey(1) & 0xFF
    
    # Press 's' to start capturing
    if key == ord('s'):
        start_capture = True

    # If capturing is started, save frames
    if start_capture:
        # Crop the frame to the ROI
        roi = frame[box_y:box_y + box_h, box_x:box_x + box_w]

        # Save the image
        img_path = os.path.join(save_dir, f"{item_name}_{img_count}.jpg")
        cv2.imwrite(img_path, roi)
        img_count += 1
        
        # Stop capturing after 100 images
        if img_count >= max_images:
            print(f"Captured {max_images} images for {item_name}.")
            break

    # Press 'q' to quit manually
    if key == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Press 's' to start capturing images of Lays_yellow.
Captured 100 images for Lays_yellow.


In [40]:
#preprocessing of images
import cv2
import os

def preprocess_images(dataset_dir, output_dir, img_size=(224, 224)):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate through each item folder
    for item_folder in os.listdir(dataset_dir):
        item_path = os.path.join(dataset_dir, item_folder)
        if os.path.isdir(item_path):
            output_item_path = os.path.join(output_dir, item_folder)
            os.makedirs(output_item_path, exist_ok=True)

            # Process each image in the folder
            for img_name in os.listdir(item_path):
                img_path = os.path.join(item_path, img_name)
                img = cv2.imread(img_path)
                
                if img is not None:
                    # Resize the image
                    resized_img = cv2.resize(img, img_size)

                    # Normalize the image (optional, to scale pixel values to 0-1)
                    normalized_img = resized_img / 255.0

                    # Save the preprocessed image
                    output_img_path = os.path.join(output_item_path, img_name)
                    cv2.imwrite(output_img_path, resized_img * 255)  # Save as uint8 (0-255)
                else:
                    print(f"Error loading image {img_path}")

# Call the function to preprocess the dataset
preprocess_images('Dataset_items', 'preprocessed_dataset', img_size=(224, 224))


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Set dataset directories
train_dir = 'Dataset_items'  # The directory with your preprocessed images

# Define image data generator for augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    validation_split=0.2,  # 20% data for validation
    rotation_range=30,     # Data augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training and validation data
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load pre-trained MobileNetV2 model (excluding the top layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding Dropout for regularization
predictions = Dense(train_data.num_classes, activation='softmax')(x)

# Define the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model (to use it as a feature extractor)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with more epochs
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,  # Start with 10 epochs, increase if needed
    verbose=1
)

# Save the trained model
model.save('grocery_item_classifier.h5')


Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.
Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.7044 - loss: 0.7051 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9985 - loss: 0.0089 - val_accuracy: 1.0000 - val_loss: 2.5136e-04
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 1.0000 - val_loss: 9.8664e-05
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9991 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 4.6455e-05
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9988 - loss: 0.0028 - val_accuracy: 1.0000 - val_loss: 2.1125e-05


In [13]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the trained classification model
model = load_model('grocery_item_classifier.h5')

# Update class labels (ensure these match your model)
class_labels = ['Lays_Masala', 'Lays_limon', 'apsara_nb', 'true_nb']  # Add other class labels as needed

# Initialize count dictionary for each class
count_dict = {label: 0 for label in class_labels}
previous_label = None  # To keep track of the previous detected label

# Confidence threshold for detecting objects
confidence_threshold = 0.8  # Set a threshold (80% in this case)

# Function to preprocess the webcam frame for prediction
def prepare_image_from_frame(frame, target_size=(224, 224)):
    img = cv2.resize(frame, target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array / 255.0  # Normalize pixel values

# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam, change if using an external camera

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to capture frame. Exiting...")
        break
    
    # Preprocess the frame for classification
    preprocessed_frame = prepare_image_from_frame(frame)

    # Make a prediction
    predictions = model.predict(preprocessed_frame)
    
    # Get the class with the highest probability
    predicted_class = np.argmax(predictions)
    
    # Ensure predicted_class index does not exceed class_labels
    if predicted_class >= len(class_labels):
        predicted_label = "Unknown class"
    else:
        predicted_label = class_labels[predicted_class]
    
    confidence = np.max(predictions)  # Get confidence in the range [0, 1]

    # Check if confidence exceeds the threshold
    if confidence < confidence_threshold:
        predicted_label = "No object detected"
        confidence = 0.0  # Set confidence to 0% if no object is detected
    else:
        confidence *= 100  # Convert to percentage for display

    # If the detected label is valid
    if predicted_label in count_dict:
        # Only increment the count if the detected label is different from the previous label
        if predicted_label != previous_label:
            count_dict[predicted_label] += 1  # Increment the count for the current label
            previous_label = predicted_label  # Update previous label to the current one
        else:
            # If the same item is detected, do nothing (skip counting)
            pass
    else:
        previous_label = None  # Reset if the prediction is invalid

    # Display the prediction and counts on the frame
    text = f"Predicted: {predicted_label} ({confidence:.2f}%)"
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display counts for each class
    y_offset = 60
    for label, count in count_dict.items():
        count_text = f"{label} Count: {count}"
        cv2.putText(frame, count_text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        y_offset += 30  # Increase the vertical position for each class count

    # Display the frame with prediction and counts
    cv2.imshow("Webcam - Grocery Item Detection", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━